In [ ]:
pip install geopandas

In [ ]:
pip install bokeh

In [ ]:
pip install country_converter

In [ ]:
pip install fuzzy_pandas

In [12]:
import pandas as pd
import geopandas as gpd
import fiona
import numpy as np

In [13]:
data_path = '~/Documents/Uni/qm2g1/data/processed_data/protests_data_filtered.csv'
protest_data =  pd.read_csv(data_path, index_col=0)

In [6]:
# function Alpha 3 country codes

import numpy as np

def do_fuzzy_search(country):
    try:
        result = pycountry.countries.search_fuzzy(country)
    except Exception:
        return np.nan
    else:
        return result[0].alpha_3

In [10]:
#Applies function on protest dataset

iso_map = {country: do_fuzzy_search(country) for country in protest_data["country"].unique()}
protest_data["country_code"] = protest_data["country"].map(iso_map)

protest_data.head()

,id,country,ccode,year,region,protest,protestnumber,startday,startmonth,startyear,...,stateresponse1,stateresponse2,stateresponse3,stateresponse4,stateresponse5,stateresponse6,stateresponse7,sources,notes,country_code
9038,7002003000,Afghanistan,700,2003,Asia,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9037,7002002001,Afghanistan,700,2002,Asia,1,1,11.0,11.0,2002.0,...,crowd dispersal,arrests,NaN,NaN,NaN,NaN,NaN,"The Advertiser, November 13, 2002;",living conditions,NaN
9036,7002001000,Afghanistan,700,2001,Asia,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9035,7002000000,Afghanistan,700,2000,Asia,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9064,7002016001,Afghanistan,700,2016,Asia,1,1,22.0,3.0,2016.0,...,ignore,NaN,NaN,NaN,NaN,NaN,NaN,Face Off Between Strongmen Exposes Afghanistan...,"In northern Afghanistan, a dispute over billb...",NaN


In [11]:
#Counts amount of protests
protest_data_grouped = protest_data.groupby(['country_code'])['protest'].sum()

protest_data_grouped = protest_data_grouped.to_frame(name="protests")
protest_data_grouped.head()

,protests
country_code,


In [ ]:
#Download shapefile from Naturalearth
url = "https://d2ad6b4ur7yvpq.cloudfront.net/naturalearth-3.3.0/ne_50m_admin_0_countries.geojson"
shapefile_data = gpd.read_file(url)[['sovereignt', 'adm0_a3', 'geometry']]

#rename colls
shapefile_data.columns = ['country', 'country_code', 'geometry']
shapefile_data.head()

In [ ]:
#merge shapefile data with protest data
merged = shapefile_data.merge(protest_data_grouped, left_on = 'country_code', right_on = 'country_code')
merged.info()

In [ ]:
import json

#Read data to json.
merged_json = json.loads(merged.to_json())

#Convert to String like object.
json_data = json.dumps(merged_json)

In [ ]:
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import brewer

#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = json_data)

#Define a sequential multi-hue color palette.
palette = brewer['YlGnBu'][8]

#Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]

#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 40)

#Define custom tick labels for color bar.
tick_labels = {'0': '0%', '5': '5%', '10':'10%', '15':'15%', '20':'20%', '25':'25%', '30':'30%','35':'35%', '40': '>40%'}

#Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
border_line_color=None,location = (0,0), orientation = 'horizontal', major_label_overrides = tick_labels)
#Create figure object.
p = figure(title = 'Share of adults who are obese, 2016', plot_height = 600 , plot_width = 950, toolbar_location = None)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
#Add patch renderer to figure. 
p.patches('xs','ys', source = geosource,fill_color = {'field' :'per_cent_obesity', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)
#Specify figure layout.
p.add_layout(color_bar, 'below')
#Display figure inline in Jupyter Notebook.
output_notebook()
#Display figure.
show(p)